In [1]:
import re
import string
import nltk
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer, PorterStemmer
from nltk.tokenize import TweetTokenizer
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"]="3"

# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation, Embedding, GRU, Input, Bidirectional
from keras.layers.embeddings import Embedding
from keras.regularizers import l2
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
def clean_text(text):
    print text
    ## Remove puncuation
    text = text.translate(string.punctuation)
    
    twtk = TweetTokenizer(strip_handles=True, reduce_len=True)
    text = " ".join([w for w in twtk.tokenize(text) if w != "" and w is not None])
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    
    text = " ".join(text)

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    text = text.split()
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)

    return text

In [3]:
# functions for cleaning
def removeStopwords(tokens):
    stops = set(stopwords.words("english"))
    stops.update(['.',',','"',"'",'?',':',';','(',')','[',']','{','}'])
    toks = [tok for tok in tokens if not tok in stops and len(tok) >= 3]
    return toks

def removeURL(text):
    newText = re.sub('http\\S+', '', text, flags=re.MULTILINE)
    return newText

def removeNum(text):
    newText = re.sub('\\d+', '', text)
    return newText

def removeHashtags(tokens):
    toks = [ tok for tok in tokens if tok[0] != '#']
#     if segment == True:
#         segTool = Analyzer('en')
#         for i, tag in enumerate(self.hashtags):
#             text = tag.lstrip('#')
#             segmented = segTool.segment(text)

    return toks

def stemTweet(tokens):
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in tokens]
    return stemmed_words


In [4]:
def processTweet(tweet, remove_swords = True, remove_url = True, remove_hashtags = True, remove_num = True, stem_tweet = True):
#     text = tweet.translate(string.punctuation)   -> to figure out what it does ?
    """
        Tokenize the tweet text using TweetTokenizer.
        set strip_handles = True to Twitter username handles.
        set reduce_len = True to replace repeated character sequences of length 3 or greater with sequences of length 3.
    """
    if remove_url:
        tweet = removeURL(tweet)
    twtk = TweetTokenizer(strip_handles=True, reduce_len=True)
    tokens = [w.lower() for w in twtk.tokenize(tweet) if w != "" and w is not None]
    if remove_hashtags:
        tokens = removeHashtags(tokens)
    if remove_swords:
        tokens = removeStopwords(tokens)
    if stem_tweet:
        tokens = stemTweet(tokens)
    text = " ".join(tokens)
    return text


In [5]:
train_data = pd.read_csv('./public_development_en/train_en.tsv',delimiter='\t',encoding='utf-8')
cleaned = train_data['text'][:5].map(lambda x: processTweet(x))
for i,t in enumerate(cleaned):
    print i,train_data['text'][i]
    print 
    print t
    print 

0 Hurray, saving us $$$ in so many ways @potus @realDonaldTrump #LockThemUp #BuildTheWall #EndDACA #BoycottNFL #BoycottNike

hurray save mani way

1 Why would young fighting age men be the vast majority of the ones escaping a war &amp; not those who cannot fight like women, children, and the elderly?It's because the majority of the refugees are not actually refugees they are economic migrants trying to get into Europe.... https://t.co/Ks0SHbtYqn

would young fight age men vast major one escap war cannot fight like women children elder major refuge actual refuge econom migrant tri get europ ...

2 @KamalaHarris Illegals Dump their Kids at the border like Road Kill and Refuse to Unite! They Hope they get Amnesty, Free Education and Welfare Illegal #FamilesBelongTogether in their Country not on the Taxpayer Dime Its a SCAM #NoDACA #NoAmnesty #SendThe

illeg dump kid border like road kill refus unit hope get amnesti free educ welfar illeg countri taxpay dime scam

3 NY Times: 'Nearly All W

In [ ]:
# tweets = train_data['text']
maxlen = 140
train_data['text'] = train_data['text'].map(lambda x: processTweet(x))
vocabulary_size = 30000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(train_data['text'])
X_train = tokenizer.texts_to_sequences(train_data['text'])
# print(sequences)
X_train = pad_sequences(X_train, maxlen=maxlen)
labels = train_data['HS']
Y_train = np_utils.to_categorical(labels, len(set(labels)))
V = len(tokenizer.word_index) + 1


l2_coef = 0.001
tweet = Input(shape=(maxlen,), dtype='int32')
x = Embedding(V, 128, input_length=maxlen, W_regularizer=l2(l=l2_coef))(tweet)
x = Bidirectional(layer=GRU(128, return_sequences=False, 
                            W_regularizer=l2(l=l2_coef),
                            b_regularizer=l2(l=l2_coef),
                            U_regularizer=l2(l=l2_coef)),
                  merge_mode='sum')(x)
x = Dense(len(set(labels)), W_regularizer=l2(l=l2_coef), activation="softmax")(x)

tweet2vec = Model(input=tweet, output=x)

tweet2vec.compile(loss='categorical_crossentropy',
                  optimizer='RMSprop',
                  metrics=['accuracy'])


tweet2vec.fit(X_train, Y_train, epochs=10, batch_size=32, validation_split=0.1)

/home/ash/aa_IRE_project/ireenv/lib/python2.7/site-packages/ipykernel_launcher.py:17: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(10687, 128, embeddings_regularizer=<keras.reg..., input_length=140)`
/home/ash/aa_IRE_project/ireenv/lib/python2.7/site-packages/ipykernel_launcher.py:21: UserWarning: Update your `GRU` call to the Keras 2 API: `GRU(128, recurrent_regularizer=<keras.reg..., bias_regularizer=<keras.reg..., kernel_regularizer=<keras.reg..., return_sequences=False)`
/home/ash/aa_IRE_project/ireenv/lib/python2.7/site-packages/ipykernel_launcher.py:23: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(2, activation="softmax", kernel_regularizer=<keras.reg...)`
/home/ash/aa_IRE_project/ireenv/lib/python2.7/site-packages/ipykernel_launcher.py:25: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`


Train on 8100 samples, validate on 900 samples
Epoch 1/10
8100/8100 [==============================] - 72s 9ms/step - loss: 0.7506 - acc: 0.7051 - val_loss: 1.1573 - val_acc: 0.5078
Epoch 2/10
8100/8100 [==============================] - 73s 9ms/step - loss: 0.5608 - acc: 0.7633 - val_loss: 1.0235 - val_acc: 0.5511
Epoch 3/10
8100/8100 [==============================] - 74s 9ms/step - loss: 0.5422 - acc: 0.7693 - val_loss: 0.8028 - val_acc: 0.6222
Epoch 4/10
8100/8100 [==============================] - 70s 9ms/step - loss: 0.5341 - acc: 0.7711 - val_loss: 0.9962 - val_acc: 0.5811
Epoch 5/10
8100/8100 [==============================] - 70s 9ms/step - loss: 0.5289 - acc: 0.7735 - val_loss: 0.8516 - val_acc: 0.6122
Epoch 6/10
8100/8100 [==============================] - 75s 9ms/step - loss: 0.5241 - acc: 0.7775 - val_loss: 0.8570 - val_acc: 0.6300
Epoch 7/10
8100/8100 [==============================] - 77s 9ms/step - loss: 0.5166 - acc: 0.7789 - val_loss: 0.9479 - val_acc: 0.5444
Epoch 8/

In [ ]:
'''
embeddings_index = dict()
f = open('./glove.twitter.27B/glove.twitter.27B.200d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

print(tokenizer.word_index.items())

# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocabulary_size, 200))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

model_glove = Sequential()
model_glove.add(Embedding(vocabulary_size, 200, input_length=50, weights=[embedding_matrix], trainable=True))
model_glove.add(Dropout(0.2))
model_glove.add(Conv1D(64, 5, activation='relu'))
model_glove.add(MaxPooling1D(pool_size=4))
model_glove.add(LSTM(200))
model_glove.add(Dense(1, activation='sigmoid'))
model_glove.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# model_glove.fit(data, np.array(labels), validation_split=0.3, epochs = 20)
'''